<a href="https://colab.research.google.com/github/Hgherzog/NoisyBatchNorm-YeoJohnsonBatchNorm/blob/main/YJBatchNorm_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is an unfinished attempt to optimize yeo johnson transform using functorch vmap. Vmap is currently not compatible with data dependendent functions and thus does not yet work.

In [ ]:
import torch, math, copy
import numpy as np
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from scipy import optimize
!pip install functorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install d2l==1.0.0-alpha1.post0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 467 kB/s 
     |████████████████████████████████| 121 kB 24.3 MB/s 
     |████████████████████████████████| 1.6 MB 77.5 MB/s 
     |████████████████████████████████| 83 kB 2.5 MB/s 


In [ ]:
#Yeo Johnson Transformation with pytorch tensors need to update to be compatible with a pytorch tensor of arbitrary dimension
#Base code acquired from scikit learn source code

from functorch import vmap
def yj_pos(x, lmbda):
  eps = torch.finfo(torch.float64).eps
  out = torch.where(abs(lmbda) < eps, torch.log(1 + x), (torch.pow(x + 1, lmbda) - 1) / lmbda)
  return out

def yj_neg(x, lmbda): 
  eps = torch.finfo(torch.float64).eps
  out = torch.where(abs(lmbda - 2) > eps, -(torch.pow(-x + 1, 2 - lmbda) - 1) / (2 - lmbda),  -torch.log(1 - x))
  return out
  
#Trying to do it with just vmap where you do vmap down the elements of the feature and then vmap that over each of the features to eliminate masking entirely
def yj_element(x, lmbda):
  eps = torch.finfo(torch.float64).eps
  # torch.where( x >=)
  # if x >= 0.0:
  #   if abs(lmbda) < eps:
  #     out = torch.log(1 + x)
  #   else:  # lmbda != 0
  #     out = (torch.pow(x + 1, lmbda) - 1) / lmbda
  # else: 
  #   if abs(lmbda - 2) > eps:
  #     out = -(torch.pow(-x + 1, 2 - lmbda) - 1) / (2 - lmbda)
  #   else:  # lmbda == 2
  #     out = -torch.log(1 - x)
  out = torch.where(x >= 0.0, yj_pos(x, lmbda), yj_neg(x, lmbda))
  return out

  
def _yeo_johnson_transform(x, lmbda):
  """Return transformed input x following Yeo-Johnson transform with
  parameter lambda, works on single feature or channel (using it as a helper fucntion for the bigger transfrom across all feature)
  """
  # out = torch.zeros_like(x)
  # pos = x >= 0  # binary mask

  # eps = torch.finfo(torch.float64).eps
  # #when x>= 0
  # if abs(lmbda) < eps:
  #   out[pos] = torch.log(1 + x[pos])
  # else:  # lmbda != 0
  #   out[pos] = (torch.pow(x[pos] + 1, lmbda) - 1) / lmbda

  # # when x < 0
  # if abs(lmbda - 2) > eps:
  #   out[~pos] = -(torch.pow(-x[~pos] + 1, 2 - lmbda) - 1) / (2 - lmbda)
  # else:  # lmbda == 2
  #   out[~pos] = -torch.log(1 +-x[~pos])
  # pos = x >= 0  # binary mask
  # print(lmbda)
  ones = torch.ones(x.size()).cuda()
  lmbda = lmbda * ones
  lmbda = lmbda.cuda()
  # print(lmbda)
  # lmbda = torch.unsqueeze(lmbda, 0)
  # print(lmbda)
  #need to make the lambda have one lambda for each element we are mapping over
  #we want to map over each element in the channel but this is a batch x (dim) x (dim) matrix
  eps = torch.finfo(torch.float64).eps
  # float_mask = torch.where(x >= 0.0, 1.0, 0.0)
  #when x>= 0
  #each feature has fixed lambda so this lambda
  #Transform entire feature for a given lambda I think the 0th or first dimension of X should access this
  yj_feature = vmap(yj_element)
  # x = torch.unsqueeze(x, 1)
  print("x shape:", x.size())
  print("lmbda shape:", lmbda.size())
  out = yj_feature(x, lmbda)
  return out

def YJ_Transform(X, lmbda):
  """Return feature or channel wise transformed input X according to the
  Yeo-Johnson transform with parameter lambda lambda shape depends on if it is being used for a fully connected or 2D convolutional Layer
  """
  # assert len(X.shape) in (2, 4)
  if len(X.shape) == 2:
    # When using a fully connected layer, transform X on the feature dimension
    #I want to lambda map the yj across the features using the corresponding
    # num_features = len(lmbda)
    # #TODO: to write this more efficiently with some sort of lambda transformation across features
    # features = []
    # for i in range(num_features):
    #   X_hat = _yeo_johnson_transform(X[i:i+1], lmbda[i])
    #   features.append(X_hat)
    #   #This should be the data such that the ith feature is transformed by the ith lambda value so that each of the features is mapped by same transform
    # X_out = torch.vstack(features)
    #vectorized version of yeo johnson
    print(lmbda)
    feature_transform = vmap(_yeo_johnson_transform, 1)
    X_out = feature_transform(X, lmbda)


  else:
  # When using a two-dimensional convolutional layer, calculate the
  # mean and variance on the channel dimension (axis=1). Here we
  # need to maintain the shape of `X`, so that the broadcasting
  # operation can be carried out later
  #need to make sure lambda has shape of number of channels for the layer if that is what the input is
    # num_channels = len(lmbda)
    # channels = []
    # #TODO: Figure out how to optimixe this to be a lambda so we don't need the for loop
    # for i in range(num_channels):
    #   # print("Channel slice for transformation", X[:,i:, :, :].size())
    #   X_hat = _yeo_johnson_transform(X[:,i:i+1, :, :], lmbda[i])
    #   channels.append(X_hat)
    # X_out = torch.cat(channels, 1)
    print(lmbda.size())
    print(X.size())
    channel_transform = vmap(_yeo_johnson_transform, 1)
    X_out = channel_transform(X, lmbda)
  return X_out


def _yeo_johnson_inverse_transform(x, lmbda):
  """Return inverse-transformed input x following Yeo-Johnson inverse
  transform with parameter lambda, across specific feture or channel
  """
  x_inv = torch.zeros_like(x)
  pos = x >= 0.0

  # when x >= 0
  eps = torch.finfo(torch.float64).eps
  if abs(lmbda[0]) < eps:
    x_inv[pos] = torch.exp(x[pos]) - 1
  else:  # lmbda != 0
    x_inv[pos] = torch.pow(x[pos] * lmbda + 1, 1 / lmbda) - 1

  # when x < 0
  if abs(lmbda[0] - 2) > eps:
    x_inv[~pos] = 1 - torch.pow(-(2 - lmbda) * x[~pos] + 1, 1 / (2 - lmbda))
  else:  # lmbda == 2
    x_inv[~pos] = 1 - torch.exp(-x[~pos])

  return x_inv

def YJINV_Transform(X, lmbda): 
  """Return feature or channel wise transformed input X according to the
  Yeo-Johnson transform with parameter lambda lambda shape depends on if it is being used for a fully connected or 2D convolutional Layer
  """
  # assert len(X.shape) in (2, 4)
  if len(X.shape) == 2:
    # When using a fully connected layer, transform X on the feature dimension
    #I want to lambda map the yj across the features using the corresponding
    num_features = len(lmbda)
    #TODO: to write this more efficiently with some sort of lambda transformation across features
    features = []
    for i in range(num_features):
      X_hat = _yeo_johnson_inverse_transform(X[i:i+1], lmbda[i])
      features.append(X_hat)
    #This should be the data such that the ith feature is transformed by the ith lambda value so that each of the features is mapped by same transform
    X_out = torch.vstack(features)

  else:
    # When using a two-dimensional convolutional layer, calculate the
    # mean and variance on the channel dimension (axis=1). Here we
    # need to maintain the shape of `X`, so that the broadcasting
    # operation can be carried out later
    #need to make sure lambda has shape of number of channels for the layer if that is what the input is
    num_channels = len(lmbda)
    channels = []
    #TODO: Figure out how to optimixe this to be a lambda so we don't need the for loop
    for i in range(num_channels):
      X_hat = _yeo_johnson_inverse_transform(X[:,i:i+1, :,:], lmbda[i])
      channels.append(X_hat)
    X_out = torch.cat(channels, 1) #I am unsure if this stacks the things back up in the way I want it too
    return X_out

#TO:DO need to update this function so that we cna do 
def _yeo_johnson_optimize(x):
  """Find and return optimal lambda parameter of the Yeo-Johnson
  transform by MLE, for observed data x. Need to do this across every feature or channel so lambda operation holds
  Like for Box-Cox, MLE is done via the brent optimizer.
  """
  #small zero like torch values 
  x_tiny = torch.finfo(torch.float64).tiny
  #make spread of lambda values from -2, 2 spread by 0.1 calculate neg log likelihood of x over these lmbda then just pick the max put it on gpu
  lmbda_vals = torch.linspace(-2, 2, 80).cuda()
  num_vals = len(lmbda_vals)

  def _neg_log_likelihood(lmbda):
    """Return the negative log likelihood of the observed data x as a
    function of lambda.
    """
    x_trans = _yeo_johnson_transform(x, lmbda)
    n_samples = x.shape[0] #may need to edit this for pytorch tensor
    x_trans_var = torch.var(x_trans) #Does this need to be attended to across dimensions?

    # Reject transformed data that would raise a RuntimeWarning in np.log
    # if x_trans_var < x_tiny:
    #   return np.inf

    log_var = torch.log(x_trans_var)
    loglike = -n_samples / 2 * log_var
    loglike += (lmbda - 1) * torch.sum(torch.sign(x) * torch.log(1 + torch.abs(x)))

    return -loglike
  NLL = vmap(_neg_log_likelihood)
  #Need to know if you want to minimize or maximize the log likelihood
  NLL_vals = NLL(lmbda_vals)
  max_ind = torch.argmax(NLL_vals)
  # max_w_storage = torch.empty_like(max_ind)
  # # Copy the values from the original tensor to the new tensor
  # max_w_storage.copy_(max_ind)
  # max_ind = max_w_storage.tolist()
  # max_ind = max_ind[0]
  opt_mask = torch.cat(torch.zeros(int(max_ind) - 1,), torch.ones(1), torch.zeros(num_vals-max_ind,))
  lmbda_hat = torch.sum(NLL_vals * opt_mask)
  # lmbda_hat = lmbda_hat.tolist()
  # lmbda_out = lmbda_hat[0]
  #lmbda_vals[0]
  # the computation of lambda is influenced by NaNs so we need to
  # get rid of them make them all 0
  # x = x[~torch.isnan(x)]
  # choosing bracket -2, 2 like for boxcox
  # return optimize.brent(_neg_log_likelihood, brack=(-2, 2)) #So optimize.brent seems to be not working lets list the options to fix this
  return lmbda_hat


In [ ]:
#network for optimizing YJ for each thing
class YJ_OPT(nn.Module):
    # `num_features`: the number of outputs for a fully connected layer
    # or the number of output channels for a convolutional layer. `num_dims`:
    # 2 for a fully connected layer and 4 for a convolutional layer
    #Does batch norm with data normalized to a log normal distribution
    def __init__(self, X_shape, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        
        self.lmbda_mle = nn.Parameter(torch.ones(X_shape))
        self.flat = nn.Flatten(0)

    def forward(self, X):
        yj_trans = vmap(YJ_Transform, 1)
        Y = yj_trans(X, self.lmbda_mle)
        Y = self.flat(Y) #Does this compute an optimal lambda across all features or channels
        return Y

In [ ]:
# model = CNNtanh(k).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
from d2l import torch as d2l
#currently testing with noise addition
def yj_batch_norm(X, gamma, beta, lmbda, moving_mean, moving_var, moving_lmbda, eps, momentum):
    # Use `is_grad_enabled` to determine whether we are in training mode
    if not torch.is_grad_enabled():
        # In prediction mode, use mean and variance obtained by moving average
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
        X_hat = YJ_Transform(X_hat, moving_lmbda)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # When using a fully connected layer, calculate the mean and
            # variance on the feature dimension
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
            # num_features = X.size(1) 
            # #want to have a little function that at each conv layer trains a lambda for all the data to be used at once
            # steps = 100
            # yj_opt = YJ_OPT(X.size(), num_features, 2).cuda()
            lmbda_g = 1.7
            lmbda_guess = lmbda_g*torch.ones(X.size())
            # for i in range(steps):
            #   optimizer.zero_grad()
            #   output = yj_opt(X)
            #   target = torch.normal(torch.zeros_like(output), torch.ones_like(output)) #should output a transformed distribution based on the learned lambda
            #   # print("output:" , output)
            #   # print("target:",target)
            #   loss = nn.MSELoss()
            #   l_lmbda = loss(output, target)
            #   l_lmbda.backward(retain_graph=True)
            #   optimizer.step()
        else:
            # When using a two-dimensional convolutional layer, calculate the
            # mean and variance on the channel dimension (axis=1). Here we
            # need to maintain the shape of `X`, so that the broadcasting
            # operation can be carried out later
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
            # num_channels = len(lmbda)
            # lmbda_mle = torch.zeros_like(lmbda)
            # for i in range(num_channels):
            #   channel = X[:,i:i+1,:, :]
            #   lmbda_mle[i] = _yeo_johnson_optimize(channel) 
            # steps = 100
            # num_features = X.size(1)
            # yj_opt = YJ_OPT(X.size(),num_features, 4).cuda()
            lmbda_g = 1.7
            lmbda_guess = lmbda_g*torch.ones(X.size())
            lmbda_guess = lmbda_guess.cuda()
            # for i in range(steps):
            #   optimizer.zero_grad()
            #   output = yj_opt(X)
            #   target = torch.normal(torch.zeros_like(output), torch.ones_like(output)) #should output a transformed distribution based on the learned lambda
            #   # print("output:" , output.size())
            #   # print("target:",target.size())
            #   loss = nn.MSELoss()
            #   l_lmbda = loss(output, target)
            #   l_lmbda.backward(retain_graph=True)
            #   optimizer.step()
        X_hat = (X - mean) / torch.sqrt(var + eps)
        print("Size before transform:", X_hat.size())
        X_hat = YJ_Transform(X, lmbda_guess)
        print("Size after transform:", X_hat.size())
        # Update the mean and variance using moving average
        moving_mean = (1.0 - momentum) * moving_mean + momentum * mean
        moving_var = (1.0 - momentum) * moving_var + momentum * var
        print(moving_lmbda.size()) 
        print(lmbda_guess.size())
        moving_lmbda = (1.0 - momentum) * moving_lmbda + momentum * lmbda_guess
        
    #Untrasform scale and shift if needed
    X_untrans = YJINV_Transform(X_hat, lmbda)
    Y =  gamma*X_untrans + beta 
    return Y, moving_mean.data, moving_var.data, moving_lmbda.data

In [ ]:
class YJ_Norm(nn.Module):
    # `num_features`: the number of outputs for a fully connected layer
    # or the number of output channels for a convolutional layer. `num_dims`:
    # 2 for a fully connected layer and 4 for a convolutional layer
    #Does batch norm with data normalized to a log normal distribution
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        #The power transform parameter is initialized to 1,
        #corresponding with the YJ identity transform
        self.lmbda = nn.Parameter(torch.ones(shape))
        # The variables that are not model parameters are initialized to 0 and
        # 1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)
        self.moving_lmbda = torch.ones(shape)

    def forward(self, X):
        # If `X` is not on the main memory, copy `moving_mean` and
        # `moving_var` to the device where `X` is located
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
            self.moving_lmbda = self.moving_lmbda.to(X.device)
        # Save the updated `moving_mean` and `moving_var`
        Y, self.moving_mean, self.moving_var , self.moving_lmbda = yj_batch_norm(
            X, self.gamma, self.beta, self.lmbda, self.moving_mean,
            self.moving_var, self.moving_lmbda, eps=1e-5, momentum=0.1)
        return Y

In [ ]:
#Load dataset
bs = 256
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
train_dataset = datasets.CIFAR10("data", train=True, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)

test_dataset = datasets.CIFAR10("data", train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), normalize
]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False)

#Device Settings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#maybe I can do an alternate load with a smaller portion of the data

In [ ]:
#Train.ipynb code 
def train(epochs, model, criterion, optimizer, train_loader, test_loader):
    epochs_and_test_err = []
    for epoch in range(epochs):
        train_err = train_epoch(model, criterion, optimizer, train_loader)
        test_err = test(model, test_loader)
        epochs_and_test_err.append((epoch, test_err))
        print('Epoch {:03d}/{:03d}, Train Error {:.2f}% || Test Error {:.2f}%'.format(epoch, epochs, train_err*100, test_err*100))
    return train_err, test_err, epochs_and_test_err
    
def train_epoch(model, criterion, optimizer, loader):
    total_correct = 0.
    total_samples = 0.
    
    for batch_idx, (data, target) in enumerate(loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        len_t = len(target)
        output = model(data)
        loss = criterion(output, target)
#I should vectorize this calculation
        for i in range(len_t):
          
          total_correct +=  1 if torch.argmax(output[i]) == target[i] else 0
          total_samples += 1

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return 1 - total_correct/total_samples

In [ ]:
#Test.ipynb code
def test(model, loader):
  total_correct = 0.
  total_samples = 0.
  model.eval()
  with torch.no_grad():
    for batch_idx, (data, target) in enumerate(loader):
      if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
      output = model(data)
#I should vectorize this transformation
      for i, val in enumerate(target):
        total_correct += 1 if torch.argmax(output[i]) == val else 0
        total_samples += 1 

  return 1 - total_correct/total_samples

In [ ]:
from torch.autograd import Variable

__all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']

#Do I need () or not
# bn_dict = {'control' : nn.BatchNorm2d, 'lognorm' : BatchLogNorm, 'adanorm' : AdaBatchNorm, 
#            'pred' : PredBatchNorm, 'gauss': G_Norm, 'mean' : MeanNorm} 

bn_dict = {'control' : nn.BatchNorm2d,'yj_norm': YJ_Norm, 'gauss': G_Norm}


def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, bn_type, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = bn_dict[bn_type](planes, 4)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = bn_dict[bn_type](planes, 4)

        self.shortcut = nn.Sequential()
        #TO_DO: make it only usable for CIFAR10
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, bn_type, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = bn_dict[bn_type](16, 4)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], bn_type, stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], bn_type, stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], bn_type, stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, bn_type, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, bn_type, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def resnet20(bn_type):
    return ResNet(BasicBlock, [3, 3, 3], bn_type)


def resnet32(bn_type):
    return ResNet(BasicBlock, [5, 5, 5], bn_type)


def resnet44(bn_type):
    return ResNet(BasicBlock, [7, 7, 7], bn_type)


def resnet56(bn_type):
    return ResNet(BasicBlock, [9, 9, 9], bn_type)


def resnet110(bn_type):
    return ResNet(BasicBlock, [18, 18, 18], bn_type)


def resnet1202(bn_type):
    return ResNet(BasicBlock, [200, 200, 200], bn_type)

#Number of parameters and layer testing
# def test_net(net):
#     import numpy as np
#     total_params = 0

#     for x in filter(lambda p: p.requires_grad, net.parameters()):
#         total_params += np.prod(x.data.numpy().shape)
#     print("Total number of params", total_params)
#     print("Total layers", len(list(filter(lambda p: p.requires_grad and len(p.data.size())>1, net.parameters()))))


# if __name__ == "__main__":
#     for net_name in __all__:
#         if net_name.startswith('resnet'):
#             print(net_name)
#             test_net(globals()[net_name]())
#             print()

In [ ]:
lr = 0.01
momentum = 0.9 
epochs = 3
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
model = ResNet(BasicBlock, [3, 3, 3], 'yj_norm').cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
# torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
train_errs, test_errs, epochs_and_test_errs = train(epochs, model, criterion, optimizer, train_loader, test_loader)

Size before transform: torch.Size([256, 16, 32, 32])
torch.Size([256, 16, 32, 32])
torch.Size([256, 16, 32, 32])
x shape: torch.Size([256, 32, 32])
lmbda shape: torch.Size([256, 32, 32])
Size after transform: torch.Size([16, 256, 32, 32])
torch.Size([1, 16, 1, 1])
torch.Size([256, 16, 32, 32])


RuntimeError: ignored

In [ ]:
#Evaluation Test Loop for different models
model = ResNet(BasicBlock, [3, 3, 3], 'control').cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
train_errs, test_errs, epochs_and_test_errs = train(epochs, model, criterion, optimizer, train_loader, test_loader)

KeyboardInterrupt: ignored

In [ ]:
#Evaluation Test Loop for different models
model = ResNet(BasicBlock, [3, 3, 3], 'yj_norm').cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
train_errs, test_errs, epochs_and_test_errs = train(epochs, model, criterion, optimizer, train_loader, test_loader)

Size before transform: torch.Size([256, 16, 32, 32])
Size after transform: torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
torch.Size([256, 512, 32])
Size before transform: torch.Size([1, 16, 256, 512])
Size after transform: torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1, 4096, 512])
torch.Size([1

RuntimeError: ignored